In [1]:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install text2num
!pip install requests


[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: C:\Users\emont\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.0/12.9 MB 682.7 kB/s eta 0:00:19
     --------------------------------------- 0.1/12.9 MB 825.8 kB/s eta 0:00:16
      --------------------------------------- 0.2/12.9 MB 2.0 MB/s eta 0:00:07
     - -------------------------------------- 0.6/12.9 MB 3.9 MB/s eta 0:00:04
     --- ------------------------------------ 1.2/12.9 MB 6.1 MB/s eta 0:00:02
     ----- ---------------------------------- 1.8/12.9 MB 7.5 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/12.9 MB 8.3 MB/s eta 0:00:02
     -------- ------------------------------- 2.6/12.9 MB 8.3 MB/s eta 0:00:02
     ---------- ----------------------------- 3.3/12.9 MB 8.7 MB/s eta 0:00:02
     ------------ --------------------------- 3.9/12.9 MB 9.3 MB/s eta 0:00:01
     ------------- -------------------------- 4.5/12.9 MB 9.5 MB/s eta 0:00:01
     -------------- ------------------------- 4.8/12.9 MB


[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: C:\Users\emont\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: C:\Users\emont\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: C:\Users\emont\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# -------------------------------------
# LIBRERÍAS Y CONFIGURACIÓN DE MODELOS
# -------------------------------------
import spacy
import json
from spacy.matcher import Matcher
from text_to_num import text2num
import requests
from datetime import datetime

nlp = spacy.load("es_core_news_sm")
matcher = Matcher(nlp.vocab)


In [3]:
# ---------------------------------------
# CONFIGURACIÓN DE INFORMACIÓN UTILIZADA
# ---------------------------------------

MESES = [
    "enero", "febrero", "marzo", "abril", "mayo", "junio",
    "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"
]

AEROLINEAS = [
    "iberia", "latam", "avianca", "copa", "american airlines", "delta", "qatar airways", "emirates", "ryanair", "jetblue",
    "aeromexico", "aeroregional", "air europa", "klm", "jetsmart", "united", "aireuropa", "air europa"
]

# --------------------------
# CONFIGURACIÓN DE MATCHERS
# --------------------------

# ---- FECHA ----
matcher.add(
    "FECHA",
    [[
        {"LIKE_NUM": True},
        {"LOWER": "de"},
        {"LOWER": {"IN": MESES}}
    ],
    [
        {"LIKE_NUM": True},
        {"LOWER": "de"},
        {"LOWER": {"IN": MESES}},
        {"LOWER": {"IN": ["de", "del"]}},
        {"LIKE_NUM": True}
    ],
    [
        {"LOWER": {"IN": ["para", "en"]}}, 
        {"LOWER": {"IN": MESES}}
    ],
    [
        {"LOWER": {"IN": MESES}}
    ]]
)

# ---- AEROLÍNEAS ----
for aerolinea in AEROLINEAS:
    pattern = [{"LOWER": palabra} for palabra in aerolinea.split()]
    matcher.add("AEROLINEA", [pattern])


In [7]:
# ---------------------
# FUNCIONES AUXILIARES
# ---------------------
def normalizar_numero(texto):
  """
    Convierte cantidades expresadas en lenguaje natural o en formato numérico
    a un valor entero. Soporta números escritos en español (ej. "veinte y cinco")
    mediante la librería text2num, garantizando escalabilidad y consistencia
    en el dato final.
  """
  texto = texto.lower().strip()
  if texto.isdigit():
      return int(texto)

  try:
      return text2num(texto, "es")
  except:
      return None

def extraer_ciudad(doc, preposicion):
    """
    Extrae el nombre completo de una ciudad, siendo compatible con ciudades
    formadas por varias palabras y con preposiciones internas, como "La Paz",
    "Buenos Aires", "Ciudad de México" o "Río de Janeiro".
    """
    ARTICULOS = {"la", "el", "los", "las"}

    for i, token in enumerate(doc):
        if token.lower_ != preposicion:
            continue

        ciudad_tokens = []
        j = i + 1

        if j < len(doc) and doc[j].lower_ in ARTICULOS:
            ciudad_tokens.append(doc[j].text)
            j += 1

        if j >= len(doc) or doc[j].pos_ != "PROPN":
            continue

        while j < len(doc) and doc[j].pos_ == "PROPN":
            ciudad_tokens.append(doc[j].text)
            j += 1

        while (
            j + 1 < len(doc)
            and doc[j].lower_ == "de"
            and doc[j + 1].pos_ == "PROPN"
        ):
            ciudad_tokens.append(doc[j].text)
            j += 1

            while j < len(doc) and doc[j].pos_ == "PROPN":
                ciudad_tokens.append(doc[j].text)
                j += 1

        if ciudad_tokens:
            return " ".join(ciudad_tokens)

    return None


def extraer_cantidad(doc):
  """
    Identifica y extrae la cantidad de boletos a partir del contexto del mensaje,
    analizando las palabras que preceden a términos como "boletos" o "pasajes".
    Permite interpretar números simples y compuestos en español
    (ej. "cuatro", "veinte y cinco") y los normaliza a valores enteros.
  """
  PALABRAS_CANTIDAD = {"boletos", "boleto", "billetes", "billete", "pasajes", "pasaje"}
  for i, token in enumerate(doc):
      if token.lower_ in PALABRAS_CANTIDAD:
          numero_tokens = []
          j = i - 1

          while j >= 0 and (
              doc[j].like_num or doc[j].lower_ == "y"
          ):
              numero_tokens.insert(0, doc[j].text)
              j -= 1

          numero_texto = " ".join(numero_tokens).strip()

          if numero_texto.isdigit():
              return int(numero_texto)

          try:
              return text2num(numero_texto, lang="es")
          except:
              return None

  return None

def extraer_ruta_sin_de(doc):
    """
    Extrae origen/destino cuando el usuario no utiliza 'de'.
    """
    STOP_ORIGEN = {
        "billete", "billetes",
        "boleto", "boletos",
        "pasaje", "pasajes",
        "comprar", "compra",
        "necesito", "quiero",
        "viaje", "viajar"
    }

    for i, token in enumerate(doc):
        if token.lower_ == "a" and i > 0 and i + 1 < len(doc):
            # --- destino hacia adelante ---
            destino_tokens = []
            k = i + 1
            while k < len(doc) and doc[k].pos_ == "PROPN":
                destino_tokens.append(doc[k].text)
                k += 1

            # --- origen hacia atrás ---
            origen_tokens = []
            j = i - 1
            while j >= 0:
                if doc[j].pos_ == "PROPN" and doc[j].lower_ not in STOP_ORIGEN:
                    origen_tokens.insert(0, doc[j].text)
                    j -= 1
                else:
                    break

            if origen_tokens and destino_tokens:
                return " ".join(origen_tokens), " ".join(destino_tokens)

    return None, None

def extract_information(text):
    """
    Función principal de procesamiento de lenguaje natural (CORREGIDA).
    Prioriza la fecha más larga encontrada para capturar el año.
    """
    doc = nlp(text)

    origen = extraer_ciudad(doc, "de")
    destino = extraer_ciudad(doc, "a")

    if origen is None or destino is None:
        o2, d2 = extraer_ruta_sin_de(doc)
        if origen is None: origen = o2
        if destino is None: destino = d2

    cantidad = extraer_cantidad(doc)
    if cantidad is None:
        cantidad = 1

    aerolinea = None
    
    candidatos_fecha = []

    for match_id, start, end in matcher(doc):
        label = nlp.vocab.strings[match_id]
        span = doc[start:end]

        if label == "FECHA":
            candidatos_fecha.append(span.text) 

        elif label == "AEROLINEA" and aerolinea is None:
            aerolinea = span.text.title()
    
    if candidatos_fecha:
        fecha = max(candidatos_fecha, key=len)
    else:
        fecha = None

    return {
        "origen": origen,
        "destino": destino,
        "fecha": fecha,
        "cantidad": cantidad,
        "aerolínea": aerolinea
    }
def normalizar_fecha_formato(fecha_texto):
    """
    Implementación de nuevo formato de fecha.
    Devuelve la fecha en formato DD-MM-AAA
    """
    if not fecha_texto: return None
    MESES = {"enero":"01","febrero":"02","marzo":"03","abril":"04","mayo":"05","junio":"06",
             "julio":"07","agosto":"08","septiembre":"09","octubre":"10","noviembre":"11","diciembre":"12"}
    texto_limpio = fecha_texto.lower()\
        .replace("para", "")\
        .replace("en", "")\
        .replace("del", " ")\
        .replace("de", " ")\
        .replace(",", " ")\
        .strip()
    partes = texto_limpio.split()  
    if not partes: return None  
    try:
        anio = datetime.now().year
        
        if not partes[0].isdigit():
            dia = 1
            mes_nombre = partes[0]
            mes_num = MESES.get(mes_nombre, "01")
            
        else:
            dia = int(partes[0])
            mes_nombre = partes[1] if len(partes) > 1 else "enero"
            mes_num = MESES.get(mes_nombre, "01")
            
            if len(partes) >= 3 and partes[2].isdigit():
                anio = int(partes[2])
                return f"{dia:02d}-{mes_num}-{anio}" 

        mes_actual = datetime.now().month
        
        if int(mes_num) < mes_actual:
            anio += 1
        elif int(mes_num) == mes_actual and dia < datetime.now().day:
            anio += 1
            
        return f"{dia:02d}-{mes_num}-{anio}"
        
    except Exception as e:
        return datetime.now().strftime("%d-%m-%Y")



In [11]:
# ------------------
# ASISTENTE VIRTUAL
# ------------------
def assistant_v1():
    print("Hola, bienvenido a BMJ.SA\n¿Cómo te puedo ayudar?")
    user_input = input(">> ")
    print(">> ",user_input)
    info = extract_information(user_input)

    if info["fecha"] is None:
      print(
          f"\nPerfecto, comienzo la búsqueda de tu viaje a {info['destino']} "
          f"desde {info['origen']} con {info['aerolínea']}."
      )
    else:
      print(
          f"\nPerfecto, comienzo la búsqueda de tu viaje a {info['destino']} "
          f"desde {info['origen']} para el {info['fecha']} "
          f"con {info['aerolínea']}."
      )

    print("\nJSON generado:")
    print(json.dumps(info, indent=4, ensure_ascii=False))


assistant_v1()

Hola, bienvenido a BMJ.SA
¿Cómo te puedo ayudar?
>>  Billete barato AirEuropa de Madrid a Sevilla

Perfecto, comienzo la búsqueda de tu viaje a Sevilla desde Madrid con Aireuropa.

JSON generado:
{
    "origen": "Madrid",
    "destino": "Sevilla",
    "fecha": null,
    "cantidad": 1,
    "aerolínea": "Aireuropa"
}


In [5]:
# ==========================================
#FUNCIÓN DE BÚSQUEDA Y VALIDACIÓN IATA
# ==========================================
def validar_ciudad(ciudad_input):
    """
    Verifica si una ciudad existe usando la API de Travelpayouts.
    Retorna el código IATA si existe o None si no se encuentra.
    """
    if not ciudad_input:
        return None

    url = "https://autocomplete.travelpayouts.com/places2"

    params = {
        "term": ciudad_input,
        "locale": "es",
        "types[]": ["city", "airport"]
    }

    try:
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()

            if data and len(data) > 0:
                resultado = data[0]
                # nombre_real = resultado.get('name')
                # pais = resultado.get('country_name')
                iata = resultado.get('code')
                # print(f"   [OK] '{ciudad_input}' identificada como: {nombre_real}, {pais} ({iata})")
                return iata
            else:
                #print(f"La ciudad '{ciudad_input}' NO existe o no fue encontrada en la base de datos.")
                return None
        else:
            print("Error al conectar con el servicio de validación.")
            return None

    except Exception as e:
        print(f"Error de conexión: {e}")
        return None

In [8]:
# ==========================================
# ASISTENTE MODIFICADO
# ==========================================
def assistant_v2():
    print("-------------------------------------------------")
    print("Hola, bienvenido a BMJ.SA")
    print("Soy un bot asistente de reservas de pasajes aéreos.")
    print("¿Cómo te puedo ayudar?")
    print("Recuerda que puedes escribir 'salir' para finalizar.")
    print("-------------------------------------------------")

    while True:
        try:
            user_input = input("\n>> ")
        except EOFError:
            break

        user_input_normalized = user_input.strip().lower()

        if user_input_normalized == "salir":
            print("Gracias por usar el asistente. Buen viaje.")
            break
        
        if not user_input_normalized:
            continue

        print("\n... Procesando solicitud ...")

        info_raw = extract_information(user_input)

        if info_raw["origen"] is None and info_raw["destino"] is None:
            print("No he entendido tu solicitud.")
            print("Soy un bot especializado en viajes. Por favor, intenta realizar tu petición nuevamente.")
            continue

        if info_raw["origen"] is None:
            print("Entendí a dónde quieres ir, pero me falta saber desde dónde sales.")
            print("Por favor, incluye el origen.")
            continue 

        if info_raw["destino"] is None:
            print("Entendí tu origen, pero no sé a dónde quieres viajar.")
            print("Por favor, incluye el destino.")
            continue

        print(f"Validando ruta: {info_raw['origen']} -> {info_raw['destino']}...")
        
        iata_from = validar_ciudad(info_raw["origen"])
        if not iata_from:
            continue 

        iata_to = validar_ciudad(info_raw["destino"])
        if not iata_to:
            continue

        fecha_fmt = normalizar_fecha_formato(info_raw["fecha"])
        
        json_final = {
            "Origen": info_raw["origen"].title(),
            "Ciudad Destino": info_raw["destino"].title(),
            "Nombre Ciudad IATA From": iata_from,
            "Nombre Ciudad IATA To": iata_to,
            "Fecha": fecha_fmt,
            "Pax": info_raw["cantidad"],
            "Aerolinea": info_raw["aerolínea"]
        }

        print("\nSOLICITUD PROCESADA CON ÉXITO")
        print(json.dumps(json_final, indent=4, ensure_ascii=False))
        print("\n¿Deseas consultar otro vuelo?")

assistant_v2()

-------------------------------------------------
Hola, bienvenido a BMJ.SA
Soy un bot asistente de reservas de pasajes aéreos.
¿Cómo te puedo ayudar?
Recuerda que puedes escribir 'salir' para finalizar.
-------------------------------------------------

... Procesando solicitud ...
Validando ruta: Madrid -> Frankfurt...

SOLICITUD PROCESADA CON ÉXITO
{
    "Origen": "Madrid",
    "Ciudad Destino": "Frankfurt",
    "Nombre Ciudad IATA From": "MAD",
    "Nombre Ciudad IATA To": "FRA",
    "Fecha": "01-09-2026",
    "Pax": 2,
    "Aerolinea": null
}

¿Deseas consultar otro vuelo?

... Procesando solicitud ...
Entendí a dónde quieres ir, pero me falta saber desde dónde sales.
Por favor, incluye el origen.

... Procesando solicitud ...
Validando ruta: Barcelona -> Roma...

SOLICITUD PROCESADA CON ÉXITO
{
    "Origen": "Barcelona",
    "Ciudad Destino": "Roma",
    "Nombre Ciudad IATA From": "BCN",
    "Nombre Ciudad IATA To": "ROM",
    "Fecha": "25-08-2026",
    "Pax": 1,
    "Aerolinea":